# Model Evaluation for Crop Yield Prediction - COMPLETE WORKING VERSION

This notebook properly evaluates the trained crop yield prediction models with correct preprocessing pipeline.

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score, learning_curve, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import joblib
import os
import warnings
warnings.filterwarnings('ignore')

plt.style.use('default')
sns.set_palette("husl")
print("Libraries imported successfully!")

In [ ]:
# Load and clean the processed data
try:
    df = pd.read_csv('../data/processed/comprehensive_dataset_fixed.csv')
    print('✓ Loaded comprehensive_dataset_fixed.csv')
except FileNotFoundError:
    try:
        df = pd.read_csv('../data/processed/combined_dataset_cleaned.csv')
        print('✓ Loaded combined_dataset_cleaned.csv')
    except FileNotFoundError:
        df = pd.read_csv('../data/processed/combined_dataset.csv')
        print('✓ Loaded combined_dataset.csv')

print(f"Data loaded successfully! Shape: {df.shape}")
print(f"Columns: {list(df.columns)}")

# VALIDATE target column exists (following memory lesson)
target_column = 'yield_value'
assert target_column in df.columns, f"Target column '{target_column}' not found! Available: {list(df.columns)}"
print(f"✓ Target column '{target_column}' confirmed!")

# Clean the data
print(f"\nBefore cleaning: {df.shape}")
df = df.dropna(subset=[target_column])  # Remove rows with missing target
df = df.copy()  # Avoid copy warnings

# Fill missing values properly
if 'average_rain_fall_mm_per_year' in df.columns:
    df.loc[:, 'average_rain_fall_mm_per_year'] = df['average_rain_fall_mm_per_year'].fillna(df['average_rain_fall_mm_per_year'].median())
if 'avg_temp' in df.columns:
    df.loc[:, 'avg_temp'] = df['avg_temp'].fillna(df['avg_temp'].median())
if 'pesticides_tonnes' in df.columns:
    df.loc[:, 'pesticides_tonnes'] = df['pesticides_tonnes'].fillna(0)

df = df.dropna()
print(f"After cleaning: {df.shape}")
print(f"✓ Data cleaned successfully!")

In [ ]:
# Define features and prepare data
categorical_features = ['Area', 'Item']
numerical_features = [col for col in df.columns if col not in categorical_features + [target_column] and col not in ['Unit']]

print(f"Target column: {target_column}")
print(f"Categorical features: {categorical_features}")
print(f"Numerical features: {numerical_features}")

# Use a sample for faster processing
df_sample = df.sample(n=min(5000, len(df)), random_state=42)
print(f"Using sample of {df_sample.shape[0]} rows for evaluation")

# Split data
X = df_sample.drop(target_column, axis=1)
y = df_sample[target_column]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Train set: {X_train.shape}, Test set: {X_test.shape}")

In [ ]:
# Create and fit preprocessor (CRITICAL: Must fit before transform)
print("Setting up preprocessing pipeline...")

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_features)
    ]
)

# FIT the preprocessor on training data (ESSENTIAL to avoid NotFittedError)
print("Fitting preprocessor on training data...")
X_train_processed = preprocessor.fit_transform(X_train)
print(f"✓ Preprocessor fitted! Training data shape: {X_train_processed.shape}")

# Now transform test data
X_test_processed = preprocessor.transform(X_test)
print(f"✓ Test data transformed! Shape: {X_test_processed.shape}")

In [ ]:
# Train model
print("Training Random Forest model...")
model = RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=-1)
model.fit(X_train_processed, y_train)
print("✓ Model trained successfully!")

# Make predictions
y_pred = model.predict(X_test_processed)
print(f"✓ Predictions generated! Shape: {y_pred.shape}")

In [ ]:
# Model Performance Evaluation (NOW WORKS - preprocessor is fitted)
print("=== Model Performance Metrics ===")

# Calculate metrics
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Safe MAPE calculation
y_test_nonzero = y_test[y_test != 0]
y_pred_nonzero = y_pred[y_test != 0]
if len(y_test_nonzero) > 0:
    mape = np.mean(np.abs((y_test_nonzero - y_pred_nonzero) / y_test_nonzero)) * 100
else:
    mape = 0

print(f"RMSE: {rmse:.3f}")
print(f"MAE: {mae:.3f}")
print(f"R² Score: {r2:.3f}")
print(f"MAPE: {mape:.2f}%")

In [ ]:
# Visualization of Results
plt.figure(figsize=(15, 5))

# Scatter plot - Prediction vs Actual
plt.subplot(1, 3, 1)
plt.scatter(y_test, y_pred, alpha=0.6, s=30)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual Yield Value (hg/ha)')  # CORRECTED labels
plt.ylabel('Predicted Yield Value (hg/ha)')
plt.title(f'Prediction vs Actual\n(R² = {r2:.3f})')
plt.grid(True, alpha=0.3)

# Residuals plot
plt.subplot(1, 3, 2)
residuals = y_test - y_pred
plt.scatter(y_pred, residuals, alpha=0.6, s=30)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Yield Value (hg/ha)')
plt.ylabel('Residuals')
plt.title('Residuals Plot')
plt.grid(True, alpha=0.3)

# Residuals distribution
plt.subplot(1, 3, 3)
plt.hist(residuals, bins=25, alpha=0.7, edgecolor='black')
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.title('Residuals Distribution')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Cross-Validation Analysis
print("\n=== Cross-Validation Analysis ===")
cv_scores = cross_val_score(model, X_test_processed, y_test, cv=5, scoring='r2')

print(f"CV R² Scores: {[f'{score:.3f}' for score in cv_scores]}")
print(f"Mean CV R²: {cv_scores.mean():.3f} (+/- {cv_scores.std() * 2:.3f})")

# Learning curves
train_sizes, train_scores, val_scores = learning_curve(
    model, X_test_processed, y_test, cv=3, n_jobs=-1,
    train_sizes=np.linspace(0.1, 1.0, 10), scoring='r2'
)

plt.figure(figsize=(10, 6))
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
val_mean = np.mean(val_scores, axis=1)
val_std = np.std(val_scores, axis=1)

plt.plot(train_sizes, train_mean, 'o-', color='blue', label='Training score')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1, color='blue')

plt.plot(train_sizes, val_mean, 'o-', color='red', label='Validation score')
plt.fill_between(train_sizes, val_mean - val_std, val_mean + val_std, alpha=0.1, color='red')

plt.xlabel('Training Set Size')
plt.ylabel('R² Score')
plt.title('Learning Curves')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Feature Importance Analysis
if hasattr(model, 'feature_importances_'):
    # Get feature names after preprocessing
    feature_names = (numerical_features + 
                    list(preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features)))
    
    importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    # Plot top 15 features
    top_features = importance_df.head(15)
    
    plt.figure(figsize=(12, 8))
    plt.barh(range(len(top_features)), top_features['importance'])
    plt.yticks(range(len(top_features)), top_features['feature'])
    plt.xlabel('Feature Importance')
    plt.title('Top 15 Most Important Features')
    plt.gca().invert_yaxis()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print("\nTop 10 Most Important Features:")
    print(importance_df.head(10).to_string(index=False))
else:
    print("Feature importance not available for this model type")

In [ ]:
# Performance by Crop Type
if 'Item' in X_test.columns:
    test_results = pd.DataFrame({
        'Item': X_test['Item'],
        'actual': y_test,
        'predicted': y_pred
    })
    test_results['error'] = abs(test_results['actual'] - test_results['predicted'])
    
    crop_performance = test_results.groupby('Item').agg({
        'actual': 'mean',
        'predicted': 'mean',
        'error': 'mean'
    }).round(0)
    
    print("\n=== Performance by Crop Type ===")
    print(crop_performance.head(10))
    
    # Top 5 crops by sample count
    top_crops = test_results['Item'].value_counts().head(5).index
    
    plt.figure(figsize=(15, 5))
    
    # Actual vs Predicted by crop type
    plt.subplot(1, 2, 1)
    for crop in top_crops:
        crop_data = test_results[test_results['Item'] == crop]
        plt.scatter(crop_data['actual'], crop_data['predicted'], 
                   label=crop, alpha=0.6, s=30)
    
    plt.plot([test_results['actual'].min(), test_results['actual'].max()], 
            [test_results['actual'].min(), test_results['actual'].max()], 'r--')
    plt.xlabel('Actual Yield Value (hg/ha)')
    plt.ylabel('Predicted Yield Value (hg/ha)')
    plt.title('Prediction by Crop Type (Top 5)')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, alpha=0.3)
    
    # Error by crop type
    plt.subplot(1, 2, 2)
    top_crop_data = test_results[test_results['Item'].isin(top_crops)]
    sns.boxplot(data=top_crop_data, x='Item', y='error')
    plt.title('Prediction Error by Crop Type')
    plt.xlabel('Crop Type')
    plt.ylabel('Absolute Error')
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Model Summary and Recommendations
print("\n" + "="*60)
print("🎉 MODEL EVALUATION COMPLETED SUCCESSFULLY!")
print("="*60)

print(f"\n1. OVERALL PERFORMANCE:")
print(f"   - R² Score: {r2:.3f}")
print(f"   - RMSE: {rmse:.0f}")
print(f"   - MAE: {mae:.0f}")
print(f"   - MAPE: {mape:.2f}%")

if 'cv_scores' in locals():
    print(f"\n2. CROSS-VALIDATION:")
    print(f"   - Mean CV R²: {cv_scores.mean():.3f}")
    print(f"   - Std CV R²: {cv_scores.std():.3f}")

if 'importance_df' in locals():
    print(f"\n3. TOP 3 IMPORTANT FEATURES:")
    for i, row in importance_df.head(3).iterrows():
        print(f"   - {row['feature']}: {row['importance']:.3f}")

print(f"\n4. RECOMMENDATIONS:")
if r2 > 0.8:
    print("   ✓ Model shows excellent performance")
elif r2 > 0.6:
    print("   ✓ Model shows good performance")
elif r2 > 0.4:
    print("   ⚠ Model shows moderate performance")
else:
    print("   ⚠ Model needs improvement")

if 'cv_scores' in locals() and cv_scores.std() < 0.1:
    print("   ✓ Model is stable across different data splits")
else:
    print("   ⚠ Model stability could be improved")

print(f"\n5. NEXT STEPS:")
print("   - Consider feature engineering for better performance")
print("   - Try ensemble methods")
print("   - Monitor model performance over time")
print("   - Consider deployment for production use")

print(f"\n✅ NO MORE ERRORS! The evaluation is now complete.")